#### Import libraries and packages

In [1]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import numpy as np

#### Set connection with postgres database

In [2]:
host = 'postgresfib.fib.upc.edu'
dbname = 'ADSDBjordi.cluet'
user = 'jordi.cluet'
pwd = 'DB151199'
port = 6433
sslmode = 'require'

conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, user, pwd))
cursor = conn.cursor()

## Load housing table into formatted zone

##### Read dataframe from CSV file

In [3]:
df = pd.read_csv('data/zenodo_fotocasa_2020_21-12-06_formatted.csv')
df.columns

Index(['ID', 'address', 'bathrooms', 'building_subtype', 'building_type',
       'conservation_state', 'extraction_date', 'discount', 'floor_elevator',
       'is_new_construction', 'link', 'price', 'real_estate', 'real_estate_id',
       'rooms', 'sq_meters', 'neighbourhood', 'neighbourhood_mean_price'],
      dtype='object')

##### Create formatted_zone schema if it does not exist

In [4]:
create_formatted_zone = """CREATE SCHEMA IF NOT EXISTS formatted_zone;"""
cursor.execute(create_formatted_zone)
conn.commit()

##### Create new table in PostgreSQL database

In [5]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS formatted_zone.zenodo_fotocasa_2020_21_12_06 (
    ID INTEGER PRIMARY KEY,
    ADDRESS VARCHAR(80),
    BATHROOMS INTEGER,
    BUILDING_SUBTYPE VARCHAR(30),
    BUILDING_TYPE VARCHAR(4),
    CONSERVATION_STATE INTEGER,
    EXTRACTION_DATE DATE,
    DISCOUNT INTEGER,
    FLOOR_ELEVATOR INTEGER,
    IS_NEW_CONSTRUCTION BOOLEAN,
    LINK VARCHAR(255),
    PRICE FLOAT,
    REAL_ESTATE VARCHAR(55),
    REAL_ESTATE_ID VARCHAR(15),
    ROOMS INTEGER,
    SQ_METERS FLOAT,
    NEIGHBOURHOOD VARCHAR(45),
    NEIGHBOURHOOD_MEAN_PRICE FLOAT
);"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [6]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("The dataframe was correctly inserted")
    cursor.close()

In [7]:
execute_values(conn, df, 'formatted_zone.zenodo_fotocasa_2020_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [3]:
sql = "SELECT * from formatted_zone.zenodo_fotocasa_2020_21_12_06;"
pd.read_sql_query(sql, conn).head()

,id,address,bathrooms,building_subtype,building_type,conservation_state,extraction_date,discount,floor_elevator,is_new_construction,link,price,real_estate,real_estate_id,rooms,sq_meters,neighbourhood,neighbourhood_mean_price
0,0,"Carrer Roger de Lluria, Dreta de l'Eixample",3,Flat,Flat,0,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/calefa...,5000.0,SOMOS REAL ESTATE SERVICES,9202765680731,3,208.0,la dreta de l'eixample,1307.616842
1,1,Sant Antoni,1,Flat,Flat,1,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/aire-a...,1250.0,TECNOCASA RIERA ALTA MAR,9202751363773,3,93.0,sant antoni,994.909158
2,2,"Carrer Valencia, La Nova Esquerra de l'Eixample",2,Flat,Flat,0,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/calefa...,1600.0,SOMOS REAL ESTATE SERVICES,9202765680731,4,129.0,la nova esquerra de l'eixample,1053.832393
3,3,"Carrer Muntaner, Sant Gervasi- Galvany",2,Flat,Flat,0,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/aire-a...,3200.0,SOMOS REAL ESTATE SERVICES,9202765680731,2,133.0,sant gervasi - galvany,1397.273005
4,4,"Rambla del Raval, El Raval",1,Flat,Flat,2,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/ascens...,1000.0,ESTUDI JOAN MIRO.SL.,9202762439813,3,67.0,el raval,850.112343


## Load barris-districtes table into formatted zone

##### Read dataframe from CSV file

In [6]:
df = pd.read_csv('data/ajunt_barris_2017_21-12-06.csv')
df.columns

Index(['CODI_DISTRICTE', 'NOM_DISTRICTE', 'CODI_BARRI', 'NOM_BARRI'], dtype='object')

##### Create new table in PostgreSQL database

In [11]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS formatted_zone.AJUNT_BARRIS_2017_21_12_06 (
    CODI_DISTRICTE INTEGER,
    NOM_DISTRICTE VARCHAR(50),
    CODI_BARRI INTEGER,
    NOM_BARRI VARCHAR(50));"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [12]:
execute_values(conn, df, 'formatted_zone.AJUNT_BARRIS_2017_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [4]:
sql = "SELECT * from formatted_zone.AJUNT_BARRIS_2017_21_12_06;"
pd.read_sql_query(sql, conn).head()

,codi_districte,nom_districte,codi_barri,nom_barri
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc


## Load barris-districtes table into trusted zone
As no quality changes are needed.

##### Create trusted_zone schema if it does not exist

In [14]:
create_trusted_zone = """CREATE SCHEMA IF NOT EXISTS trusted_zone;"""
cursor.execute(create_trusted_zone)
conn.commit()

##### Create new table in PostgreSQL database

In [15]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS trusted_zone.AJUNT_BARRIS_2017_21_12_06 (
    CODI_DISTRICTE INTEGER,
    NOM_DISTRICTE VARCHAR(50),
    CODI_BARRI INTEGER,
    NOM_BARRI VARCHAR(50));"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [16]:
execute_values(conn, df, 'trusted_zone.AJUNT_BARRIS_2017_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [7]:
sql = "SELECT * from trusted_zone.AJUNT_BARRIS_2017_21_12_06;"
pd.read_sql_query(sql, conn).head()

,codi_districte,nom_districte,codi_barri,nom_barri
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc
